In [ ]:
!pip install -U vllm fastapi uvicorn nest-asyncio pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/6

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("36zD9P6h7kEM5VlTYOzGu4POhLL_5zmvV1iJpRH59JCT6wZA2")

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from vllm import LLM, SamplingParams

MODEL_NAME = "Qwen/Qwen3-30B-A3B-Instruct-2507"

llm = LLM(
    model=MODEL_NAME,
    tensor_parallel_size=1,
    dtype="auto",
    trust_remote_code=True,
    max_model_len=32768
)

sampling_params = SamplingParams(
    temperature=0.0,
    max_tokens=10
)

print("vLLM model loaded")


INFO 12-28 18:17:15 [utils.py:253] non-default args: {'trust_remote_code': True, 'max_model_len': 32768, 'disable_log_stats': True, 'model': 'Qwen/Qwen3-30B-A3B-Instruct-2507'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

INFO 12-28 18:17:37 [model.py:514] Resolved architecture: Qwen3MoeForCausalLM
INFO 12-28 18:17:37 [model.py:1661] Using max model len 32768
INFO 12-28 18:17:39 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

WARNING 12-28 18:17:44 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-28 18:23:48 [llm.py:360] Supported tasks: ['generate']
vLLM model loaded


In [ ]:
CRITERIA = """
Score 1: The answer is completely unrelated to the reference.
Score 3: The answer has minor relevance but does not align with the reference.
Score 5: The answer has moderate relevance but contains inaccuracies.
Score 7: The answer aligns with the reference but has minor omissions.
Score 10: The answer is completely accurate and aligns perfectly with the reference.
Only respond with a numerical score.
"""


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="LLM Needle Evaluation Server")

class EvalRequest(BaseModel):
    text: str
    needle: str

class EvalResponse(BaseModel):
    score: int


In [ ]:
def build_prompt(text: str, needle: str) -> str:
    return f"""<|im_start|>system
You are an evaluator.<|im_end|>
<|im_start|>user
REFERENCE (needle):
{needle}

TEXT TO EVALUATE:
{text}

Evaluate how well the TEXT matches the REFERENCE according to the following criteria:

{CRITERIA}

Respond with ONLY a number.<|im_end|>
<|im_start|>assistant
"""


In [ ]:
def evaluate(text: str, needle: str) -> int:
    prompt = build_prompt(text, needle)

    outputs = llm.generate([prompt], sampling_params)
    output_text = outputs[0].outputs[0].text.strip()

    for token in output_text.split():
        if token.isdigit():
            return max(1, min(10, int(token)))

    return 1


In [ ]:
@app.post("/evaluate", response_model=EvalResponse)
def evaluate_endpoint(req: EvalRequest):
    score = evaluate(req.text, req.needle)
    return {"score": score}


In [ ]:
import uvicorn
import asyncio
from pyngrok import ngrok

PORT = 8000

public_url = ngrok.connect(PORT)
print("🚀 Public URL:", public_url)

# uvicorn config
config = uvicorn.Config(
    app,
    host="0.0.0.0",
    port=PORT,
    log_level="info"
)

server = uvicorn.Server(config)

# запускаем сервер внутри текущего event loop
async def start_server():
    await server.serve()

asyncio.create_task(start_server())


🚀 Public URL: NgrokTunnel: "https://undatable-sympathetic-inez.ngrok-free.dev" -> "http://localhost:8000"


<Task pending name='Task-1' coro=<start_server() running at /tmp/ipython-input-3470947436.py:21>>